# RAG Pipeline with Opik Tracking

This notebook demonstrates how to use Comet ML's Opik for:
- 📊 LLM observability
- 📈 Performance tracking
- 🔍 Debugging RAG pipelines
- ✅ Evaluation with metrics

## Setup

First, install Opik:
```bash
pip install opik
```

Then configure Opik (one-time setup):
```bash
opik configure
```

In [ ]:
import sys
from pathlib import Path

project_root = str(Path.cwd().parent.parent)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

## Import Modules

In [ ]:
from src.utils.config import settings
from src.ingestion.parsers.parsers import DoclingParser
from src.ingestion.chunking.get_chunker import get_chunker
from src.ingestion.embedding.get_embbedder import get_embedder
from src.ingestion.vector_store.stores import ChromaStore

# Import generation modules
from generation import (
    OllamaGenerator,
    QueryAnswerer,
    MultiQueryConstructor,
)

# Import Opik-tracked pipelines
try:
    from generation import (
        OpikTracker,
        OpikTrackedSimpleRAGPipeline,
        OpikTrackedMultiQueryRAGPipeline,
    )
    OPIK_AVAILABLE = True
    print("✅ Opik available")
except ImportError:
    OPIK_AVAILABLE = False
    print("⚠️  Opik not installed. Run: pip install opik")

## Step 1-4: Standard RAG Setup

(Same as before: parse, chunk, embed, store)

In [ ]:
# Parse PDF
pdf_path = "../../data/Word2Vec.pdf"
parser = DoclingParser()
parsed_doc = parser.parse(pdf_path=pdf_path)
print(f"✅ Parsed {parsed_doc.metadata.nbr_pages} pages")

# Chunk
chunker = get_chunker()
chunked_doc = chunker.chunk(parsed_doc)
print(f"✅ Created {len(chunked_doc)} chunks")

# Embed
embedder = get_embedder()
embeddings = embedder.embed_chunk(chunks=chunked_doc)
print(f"✅ Embedded {len(embeddings)} chunks")

# Store
vector_store = ChromaStore(settings.vector_store)
print("🧹 Clearing old data...")
vector_store.clear()
vector_store.ingest(embch=embeddings)
print(f"✅ Stored {vector_store.count()} documents")

## Step 5: Initialize Opik Tracker

In [ ]:
if OPIK_AVAILABLE:
    # Initialize Opik tracker
    opik_tracker = OpikTracker(
        project_name="word2vec-rag",
        enabled=True  # Set to False to disable tracking
    )
else:
    opik_tracker = None
    print("⚠️  Continuing without Opik tracking")

## Step 6: Initialize RAG Components with Opik

In [ ]:
# Initialize generator and answerer
generator = OllamaGenerator(settings.llm, auto_setup=True)
answerer = QueryAnswerer(generator)

if OPIK_AVAILABLE and opik_tracker:
    # Opik-tracked pipelines
    simple_rag = OpikTrackedSimpleRAGPipeline(
        vector_store=vector_store,
        answerer=answerer,
        opik_tracker=opik_tracker
    )
    
    query_constructor = MultiQueryConstructor(generator)
    multi_rag = OpikTrackedMultiQueryRAGPipeline(
        vector_store=vector_store,
        answerer=answerer,
        query_constructor=query_constructor,
        opik_tracker=opik_tracker
    )
    
    print("✅ RAG pipelines ready with Opik tracking!")
else:
    # Fallback to non-tracked pipelines
    from generation.pipeline import SimpleRAGPipeline, MultiQueryRAGPipeline
    
    simple_rag = SimpleRAGPipeline(
        vector_store=vector_store,
        answerer=answerer
    )
    
    query_constructor = MultiQueryConstructor(generator)
    multi_rag = MultiQueryRAGPipeline(
        vector_store=vector_store,
        answerer=answerer,
        query_constructor=query_constructor
    )
    
    print("✅ RAG pipelines ready (without Opik tracking)")

## Test 1: Simple RAG with Tracking

In [ ]:
print("="*60)
print("SIMPLE RAG TEST (with Opik tracking)")
print("="*60)

question = "What is Word2Vec?"
answer = simple_rag.query(question, top_k=3)

print(f"\n📝 Answer:\n{answer}")

if OPIK_AVAILABLE:
    print("\n✅ Check Opik dashboard for detailed traces!")

## Test 2: Multi-Query RAG with Tracking

In [ ]:
print("="*60)
print("MULTI-QUERY RAG TEST (with Opik tracking)")
print("="*60)

question = "How are embeddings constructed?"
answer = multi_rag.query(question, top_k=5)

print(f"\n📝 Answer:\n{answer}")

if OPIK_AVAILABLE:
    print("\n✅ Check Opik dashboard for:")
    print("   - Query expansion traces")
    print("   - Retrieval metrics")
    print("   - Generation details")

## Test 3: Batch Evaluation with Opik

In [ ]:
if OPIK_AVAILABLE and opik_tracker:
    # Create test dataset
    test_questions = [
        "What is Word2Vec?",
        "How does CBOW differ from Skip-gram?",
        "What are the training objectives?",
    ]
    
    print("🧪 Running batch evaluation...")
    results = []
    
    for i, question in enumerate(test_questions, 1):
        print(f"\n[{i}/{len(test_questions)}] {question}")
        answer = simple_rag.query(question, top_k=3)
        print(f"Answer: {answer[:100]}...")
        results.append({
            "question": question,
            "answer": answer
        })
    
    print("\n✅ Batch evaluation complete!")
    print("📊 View results in Opik dashboard")
else:
    print("⚠️  Batch evaluation requires Opik")

## What You'll See in Opik Dashboard:

1. **Traces**: Complete execution flow
   - Query input
   - Query expansion (multi-query)
   - Document retrieval
   - Answer generation

2. **Metrics**:
   - Latency per step
   - Number of documents retrieved
   - Average relevance scores
   - Token usage

3. **Evaluation** (if configured):
   - Answer relevance
   - Context precision/recall
   - Hallucination detection

4. **Comparison**:
   - Simple vs Multi-query performance
   - Different top_k values
   - Model variations